In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
import os.path
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials

# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

# Machine learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.preprocessing import quantile_transform

# Styles
sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [2]:
# Extraemos los dataframes de nuestros files
df_tracks = pd.read_pickle('../sources/tracks.pickle')
df_af = pd.read_pickle('../sources/audio_features.pickle')
df_aa = pd.read_pickle('../sources/audio_analysis.pickle') \
    .set_index(['id','start']) \
    .sort_values(by=['id', 'start'], ascending=True, na_position='first')

In [3]:
# Generamos un dataframe combinando la data de tracks y de audio features
df_merged = df_tracks.merge(
    df_af, 
    on='id', 
    how='left'
)
df_merged = df_merged.drop(
    [
     'type_x',
     'type_y',
     'uri_x',
     'uri_y',
     'track_href',
     'analysis_url',
     'href',
     'preview_url',
     'external_ids',
     'duration_ms_y'
    ], 
    1
)

df_merged.rename(columns = {"duration_ms_x": "duration_ms"}, inplace = True)

In [4]:
# Extraemos para cada registro la variable release_date y la asignamos a una nueva columna en nuestro dataframe.

# Función que generamos para formatear la fecha
def date_formator(date):
    if '-' in date:
        year = pd.to_datetime(date, format = '%Y-%m-%d').year
        return int(year)
    elif int(date)>0:
        year = pd.to_datetime(date, format = '%Y').year
        return int(year)
    return None


df_merged['release_date'] = df_merged['album'].map(lambda x: x['release_date'])
df_merged['release_date'] = df_merged['release_date'].apply(date_formator)
df_merged['release_date'].head()

id
4d3XHYFFuYYzxWr2cJ6yQl    1991.0
3X3p3u03P8eFL8WTH0oaaU    1958.0
4KlXeaheot0OI9PoOWspvZ    2014.0
3j76McoUI18gDoqxYE14Bu    2014.0
4zxd4tiXPlWMqoJltbVTbE    1959.0
Name: release_date, dtype: float64

In [5]:
# Obtengo ids de album
df_merged['album_id'] = df_merged['album'].map(lambda x: x['id'])

# Agrego artistas y género
df_merged[["artists","genre"]] =  df_merged.loc[:,["artists","genre"]] 

In [6]:
# Vemos cuales son nuestras columnas
df_merged.columns.values

array(['album', 'artists', 'available_markets', 'disc_number',
       'duration_ms', 'explicit', 'external_urls', 'is_local', 'name',
       'popularity', 'track_number', 'genre', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'release_date', 'album_id'], dtype=object)

In [7]:
df_merged.sample(3).T

id,0PbkNsBkzlItqOC85tXCjH,4ASq8Q9pF05YGHxqL0PveM,6Sy9BUbgFse0n0LPA5lwy5
album,"{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external..."
artists,Sied Van Riel,Samuel Barber,Darude
available_markets,[],"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...","[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
disc_number,1,1,1
duration_ms,469975,278426,225493
explicit,False,False,False
external_urls,{'spotify': 'https://open.spotify.com/track/0P...,{'spotify': 'https://open.spotify.com/track/4A...,{'spotify': 'https://open.spotify.com/track/6S...
is_local,False,False,False
name,Rush,"Piano Sonata, Op. 26: IV. Fuga: Allegro con sp...",Sandstorm
popularity,0,5,72


Desdoblamos los features de pitches y timbre en distintas columnas

In [8]:
df_m2 = df_aa

pitches_nombre = ['p00_C','p01_C#','p02_D','p03_D#','p04_E','p05_F',
                 'p06_F#','p07_G','p08_G#','p09_A','p10_A#','p11_B']
df1 = pd.DataFrame(df_m2['pitches'].tolist(), columns=pitches_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

timbre_nombre = ['t00','t01','t02','t03','t04','t05',
                 't06','t07','t08','t09','t10','t11']
df1 = pd.DataFrame(df_m2['timbre'].tolist(), columns=timbre_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1).drop(['pitches','timbre'],1)

df1 = None
df_m2.head(3).T

id                00At7PWydsvg7g5xgaYan9                      
start                            0.00000    0.24381    0.41873
duration                         0.24381    0.17492    0.17329
confidence                       0.00000    1.00000    1.00000
loudness_start                 -60.00000  -60.00000  -34.27900
loudness_max_time                0.00000    0.01436    0.01554
loudness_max                   -60.00000  -12.45700  -12.72800
loudness_end                     0.00000    0.00000    0.00000
p00_C                            1.00000    0.12100    0.03200
p01_C#                           1.00000    0.99600    1.00000
p02_D                            1.00000    0.08000    0.06000
p03_D#                           1.00000    0.10400    0.05900
p04_E                            1.00000    1.00000    0.36800
p05_F                            1.00000    0.22300    0.08400
p06_F#                           1.00000    0.04300    0.03600
p07_G                            1.00000    0.04700    0.04100
p08_G#                           1.00000    0.17600    0.10600
p09_A                            1.00000    0.42200    0.38000
p10_A#                           1.00000    0.12900    0.11200
p11_B                            1.00000    0.06200    0.08700
t00                              0.00000   37.64100   38.17100
t01                            171.13000  -77.64300  -17.74100
t02                              9.46900  -30.61300  -22.79800
t03                            -28.48000  137.46200  149.22300
t04                             57.49100   36.96000   31.83300
t05                            -50.06700  -51.02400  -53.44600
t06                             14.83300  -18.74500  -36.48500
t07                              5.35900  -28.82200  -39.47900
t08                            -27.22800   -7.17000  -25.79500
t09                              0.97300   -1.99400   -1.90300
t10                            -10.64000  -15.69500  -21.90800
t11                             -7.22800    9.41100   15.93400

Extraemos los datos de MEDIANA + IQR (RANGO INTERCUARTIL)

In [9]:
grouped = df_m2.drop(['duration'],1).groupby(level='id')
medianas = grouped.quantile(.5)
iqr = grouped.quantile(.75)-grouped.quantile(.25)
medias = grouped.mean()
desvios = grouped.std()

In [10]:
df_pitches_timbres_medias_std = pd.merge(medias, desvios, on='id', suffixes=('_media', '_std'))
df_pitches_timbres_medianas_iqr = pd.merge(medianas, iqr, on='id', suffixes=('_mediana', '_iqr'))

Y ahora hacemos lo mismo para la derivada de las funciones

In [12]:
df_m2.head(10)

duration  confidence  loudness_start  \
id                     start                                           
00At7PWydsvg7g5xgaYan9 0.00000   0.24381       0.000         -60.000   
                       0.24381   0.17492       1.000         -60.000   
                       0.41873   0.17329       1.000         -34.279   
                       0.59202   0.17351       1.000         -32.474   
                       0.76553   0.17569       1.000         -32.786   
                       0.94122   0.16617       1.000         -29.257   
                       1.10739   0.17483       1.000         -28.415   
                       1.28222   0.15841       1.000         -31.659   
                       1.44063   0.18553       1.000         -28.176   
                       1.62617   0.17361       0.973         -27.416   

                                loudness_max_time  loudness_max  loudness_end  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000            0.00000       -60.000           0.0   
                       0.24381            0.01436       -12.457           0.0   
                       0.41873            0.01554       -12.728           0.0   
                       0.59202            0.01059       -10.984           0.0   
                       0.76553            0.01254        -9.480           0.0   
                       0.94122            0.01003       -14.634           0.0   
                       1.10739            0.01421       -10.546           0.0   
                       1.28222            0.01300        -7.907           0.0   
                       1.44063            0.02452        -8.306           0.0   
                       1.62617            0.01273       -13.883           0.0   

                                p00_C  p01_C#  p02_D  p03_D#  ...     t02  \
id                     start                                  ...           
00At7PWydsvg7g5xgaYan9 0.00000  1.000   1.000  1.000   1.000  ...   9.469   
                       0.24381  0.121   0.996  0.080   0.104  ... -30.613   
                       0.41873  0.032   1.000  0.060   0.059  ... -22.798   
                       0.59202  0.286   1.000  0.237   0.277  ...   9.159   
                       0.76553  0.064   0.878  0.028   0.129  ... -50.706   
                       0.94122  0.153   0.890  0.070   0.057  ... -31.294   
                       1.10739  0.209   1.000  0.121   0.116  ...   4.682   
                       1.28222  0.056   1.000  0.061   0.114  ... -36.092   
                       1.44063  0.059   1.000  0.041   0.096  ... -20.439   
                       1.62617  0.406   1.000  0.292   0.388  ... -19.722   

                                    t03     t04     t05     t06     t07  \
id                     start                                              
00At7PWydsvg7g5xgaYan9 0.00000  -28.480  57.491 -50.067  14.833   5.359   
                       0.24381  137.462  36.960 -51.024 -18.745 -28.822   
                       0.41873  149.223  31.833 -53.446 -36.485 -39.479   
                       0.59202  164.086  33.475 -83.620 -51.036 -23.505   
                       0.76553  130.054  -4.597 -82.764 -32.596 -48.432   
                       0.94122   78.563  17.387 -83.835 -39.964 -32.097   
                       1.10739  152.144  66.114 -68.732 -40.880 -20.270   
                       1.28222  129.860  38.891 -85.234 -36.440 -35.347   
                       1.44063  127.598  60.108  35.909   9.787 -46.725   
                       1.62617  113.905  40.764 -74.562 -29.151  -4.066   

                                   t08     t09     t10     t11  
id                     start                                    
00At7PWydsvg7g5xgaYan9 0.00000 -27.228   0.973 -10.640  -7.228  
                       0.24381  -7.170  -1.994 -15.695   9.411  
                       0.41873 -25.795  -1.903 -21.908  15.934  
                       0.59202 -19.636   8.

In [13]:
dff = df_m2
shifted = dff.groupby(level="id").shift(-1)
dif = shifted.drop(['duration'],1).subtract(dff.drop(['duration'],1), fill_value=0)
diferencias = dif.divide(dff['duration'], axis="index")
diferencias.head(10)

confidence  loudness_start  loudness_max_time  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000    4.101554        0.000000           0.058898   
                       0.24381    0.000000      147.044363           0.006746   
                       0.41873    0.000000       10.416066          -0.028565   
                       0.59202    0.000000       -1.798167           0.011239   
                       0.76553    0.000000       20.086516          -0.014287   
                       0.94122    0.000000        5.067100           0.025155   
                       1.10739    0.000000      -18.555168          -0.006921   
                       1.28222    0.000000       21.987248           0.072723   
                       1.44063   -0.145529        4.096373          -0.063548   
                       1.62617    0.155521      -20.782213          -0.006336   

                                loudness_max  loudness_end     p00_C  \
id                     start                                           
00At7PWydsvg7g5xgaYan9 0.00000    195.000205           0.0 -3.605266   
                       0.24381     -1.549280           0.0 -0.508804   
                       0.41873     10.064054           0.0  1.465751   
                       0.59202      8.668088           0.0 -1.279465   
                       0.76553    -29.335762           0.0  0.506574   
                       0.94122     24.601312           0.0  0.337004   
                       1.10739     15.094663           0.0 -0.875136   
                       1.28222     -2.518780           0.0  0.018938   
                       1.44063    -30.059829           0.0  1.870317   
                       1.62617     19.382524           0.0 -1.774091   

                                  p01_C#     p02_D    p03_D#     p04_E  ...  \
id                     start                                            ...   
00At7PWydsvg7g5xgaYan9 0.00000 -0.016406 -3.773430 -3.674993  0.000000  ...   
                       0.24381  0.022868 -0.114338 -0.257260 -3.613080  ...   
                       0.41873  0.000000  1.021409  1.258007  1.881240  ...   
                       0.59202 -0.703130 -1.204542 -0.852977  1.763587  ...   
                       0.76553  0.068302  0.239057 -0.409813 -2.942683  ...   
                       0.94122  0.661973  0.306915  0.355058 -0.096287  ...   
                       1.10739  0.000000 -0.343191 -0.011440  0.228794  ...   
                       1.28222  0.000000 -0.126255 -0.113629  0.782779  ...   
                       1.44063  0.000000  1.352881  1.573869  1.040263  ...   
                       1.62617  0.000000 -1.123207 -1.866252 -3.196820  ...   

                                       t02         t03         t04  \
id                     start                                         
00At7PWydsvg7g5xgaYan9 0.00000 -164.398507  680.620155  -84.209015   
                       0.24381   44.677567   67.236451  -29.310542   
                       0.41873  184.413411   85.769519    9.475446   
                       0.59202 -345.023342 -196.138551 -219.422512   
                       0.76553  110.490068 -293.078718  125.129489   
                       0.94122  216.501173  442.805561  293.235843   
                       1.10739 -233.220843 -127.460962 -155.711262   
                       1.28222   98.813206  -14.279402  133.937251   
                       1.44063    3.864604  -73.804776 -104.263461   
                       1.62617  324.935200  381.850124  240.176257   

                                       t05         t06         t07  \
id                     start                                         
00At7PWydsvg7g5xgaYan9 0.00000   -3.925188 -137.721997 -140.195234   
                       0.24381  -13.846330 -101.417791  -60.924994   
                       0.41873 -174.124300  -83.969069   92.180737   
                       0.59202    4.933433  106.276295

In [14]:
grouped_dif = diferencias.groupby(level='id')
medianas = grouped_dif.quantile(.5)
iqr = grouped_dif.quantile(.75)-grouped.quantile(.25)
medias = grouped_dif.mean()
desvios = grouped_dif.std()

In [15]:
dif_pitches_timbres_medias_std = pd.merge(medias, desvios, on='id', suffixes=('_dif_media', '_dif_std'))
dif_pitches_timbres_medianas_iqr = pd.merge(medianas, iqr, on='id', suffixes=('_dif_mediana', '_dif_iqr'))
len(dif_pitches_timbres_medias_std)

4177

Y lo unimos todo al otro dataset

In [46]:
df_merged2 = df_merged.select_dtypes(['number']).drop(['disc_number','track_number','release_date'],1)

df_merge_af = df_pitches_timbres_medianas_iqr.merge(
    dif_pitches_timbres_medianas_iqr,
    on='id', 
    how='inner'
)

df_todo = df_merged2.add_prefix('af_').merge(
    df_merge_af.add_prefix('aa_'),
    on='id', 
    how='inner'
)
df_todo['genre']=df_merged['genre']

list(df_todo.columns)

['af_duration_ms',
 'af_popularity',
 'af_danceability',
 'af_energy',
 'af_key',
 'af_loudness',
 'af_mode',
 'af_speechiness',
 'af_acousticness',
 'af_instrumentalness',
 'af_liveness',
 'af_valence',
 'af_tempo',
 'af_time_signature',
 'aa_confidence_mediana',
 'aa_loudness_start_mediana',
 'aa_loudness_max_time_mediana',
 'aa_loudness_max_mediana',
 'aa_loudness_end_mediana',
 'aa_p00_C_mediana',
 'aa_p01_C#_mediana',
 'aa_p02_D_mediana',
 'aa_p03_D#_mediana',
 'aa_p04_E_mediana',
 'aa_p05_F_mediana',
 'aa_p06_F#_mediana',
 'aa_p07_G_mediana',
 'aa_p08_G#_mediana',
 'aa_p09_A_mediana',
 'aa_p10_A#_mediana',
 'aa_p11_B_mediana',
 'aa_t00_mediana',
 'aa_t01_mediana',
 'aa_t02_mediana',
 'aa_t03_mediana',
 'aa_t04_mediana',
 'aa_t05_mediana',
 'aa_t06_mediana',
 'aa_t07_mediana',
 'aa_t08_mediana',
 'aa_t09_mediana',
 'aa_t10_mediana',
 'aa_t11_mediana',
 'aa_confidence_iqr',
 'aa_loudness_start_iqr',
 'aa_loudness_max_time_iqr',
 'aa_loudness_max_iqr',
 'aa_loudness_end_iqr',
 'aa_p

In [47]:
df_todo.to_pickle("../sources/df_todo_0_sin_escalar.pickle")

df_todo.head(3).describe().T.head(30)

,count,mean,std,min,25%,50%,75%,max
af_duration_ms,3.0,347470.666667,2979.286044,344066.00000,346406.000000,348746.000000,349173.000000,349600.00000
af_popularity,3.0,36.333333,20.231988,13.00000,30.000000,47.000000,48.000000,49.00000
af_danceability,3.0,0.631333,0.153155,0.48700,0.551000,0.615000,0.703500,0.79200
af_energy,3.0,0.216667,0.078768,0.14500,0.174500,0.204000,0.252500,0.30100
af_key,3.0,7.000000,2.000000,5.00000,6.000000,7.000000,8.000000,9.00000
af_loudness,3.0,-14.890000,4.491317,-18.67000,-17.372500,-16.075000,-13.000000,-9.92500
af_mode,3.0,0.666667,0.577350,0.00000,0.500000,1.000000,1.000000,1.00000
af_speechiness,3.0,0.045667,0.005853,0.04090,0.042400,0.043900,0.048050,0.05220
af_acousticness,3.0,0.764667,0.189381,0.54600,0.709000,0.872000,0.874000,0.87600
af_instrumentalness,3.0,0.675000,0.253746,0.38200,0.601500,0.821000,0.821500,0.82200


In [70]:
from sklearn.preprocessing import MinMaxScaler

df_para_escalar = df_todo.select_dtypes(['number'])

col_af = [col for col in df_para_escalar if col.startswith('af')]
col_aa = [col for col in df_para_escalar if col.startswith('aa')]

scaler = MinMaxScaler(feature_range=(0,1))
scaled_af = pd.DataFrame(scaler.fit_transform(df_para_escalar.loc[:, col_af]),columns=col_af,index=df_para_escalar.index)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_aa = pd.DataFrame(scaler.fit_transform(df_para_escalar.loc[:, col_aa]),columns=col_aa,index=df_para_escalar.index)

scaled_aa.head()

df_escalado = pd.concat([scaled_af, scaled_aa], axis=1)
df_escalado['genre']=df_todo['genre']

df_escalado.to_pickle("../sources/df_todo_1_MinMaxScaled.pickle")

df_escalado.describe().T.head(30)

,count,mean,std,min,25%,50%,75%,max
af_duration_ms,4082.0,0.071738,0.044974,0.0,0.045713,0.062317,0.085867,1.0
af_popularity,4082.0,0.220605,0.240983,0.0,0.000000,0.162791,0.383721,1.0
af_danceability,4082.0,0.467091,0.196402,0.0,0.312896,0.476216,0.613901,1.0
af_energy,4082.0,0.552175,0.340881,0.0,0.217982,0.578913,0.897736,1.0
af_key,4082.0,0.471182,0.321789,0.0,0.181818,0.454545,0.727273,1.0
af_loudness,4082.0,0.736707,0.164114,0.0,0.641785,0.784772,0.864777,1.0
af_mode,4082.0,0.644047,0.478860,0.0,0.000000,1.000000,1.000000,1.0
af_speechiness,4082.0,0.077912,0.066009,0.0,0.044224,0.054492,0.081884,1.0
af_acousticness,4082.0,0.416802,0.420862,0.0,0.001923,0.218876,0.908635,1.0
af_instrumentalness,4082.0,0.387383,0.394439,0.0,0.000741,0.211616,0.841414,1.0


# ESCALAMIENTO DE FEATURES

Y LO NORMALIZAMOS (al menos hasta donde puede hacerse de una manera sencilla)

Primero con MinMaxScaler

Y despues con Normal Quantiles

In [69]:
from sklearn.preprocessing import QuantileTransformer

df_para_escalar = df_todo.select_dtypes(['number'])

col_af = [col for col in df_para_escalar if col.startswith('af')]
col_aa = [col for col in df_para_escalar if col.startswith('aa')]

scaler = QuantileTransformer(n_quantiles=100, output_distribution = 'normal', random_state = 0)
scaled_af = pd.DataFrame(scaler.fit_transform(df_para_escalar.loc[:, col_af]),columns=col_af,index=df_para_escalar.index)
scaler = QuantileTransformer(n_quantiles=100, output_distribution = 'normal', random_state = 0)
scaled_aa = pd.DataFrame(scaler.fit_transform(df_para_escalar.loc[:, col_aa]),columns=col_aa,index=df_para_escalar.index)

scaled_aa.head()

df_escalado = pd.concat([scaled_af, scaled_aa], axis=1)
df_escalado['genre']=df_todo['genre']

df_escalado.to_pickle("../sources/df_todo_1_Normalized.pickle")

df_escalado.describe().T.head(30)


,count,mean,std,min,25%,50%,75%,max
af_duration_ms,4082.0,-0.001851,9.926864e-01,-5.199338,-0.678479,0.000789,0.672820,5.199338
af_popularity,4082.0,-1.769321,2.929961e+00,-5.199338,-5.199338,0.012660,0.666564,5.199338
af_danceability,4082.0,0.000725,9.962546e-01,-5.199338,-0.673879,-0.000275,0.671317,5.199338
af_energy,4082.0,-0.000224,1.003355e+00,-5.199338,-0.674490,-0.001582,0.675625,5.199338
af_key,4082.0,-0.173515,2.295738e+00,-5.199338,-0.619855,-0.050661,0.635270,5.199338
af_loudness,4082.0,-0.001218,9.945427e-01,-5.199338,-0.674745,0.000878,0.676413,5.199338
af_mode,4082.0,1.497898,4.979507e+00,-5.199338,-5.199338,5.199338,5.199338,5.199338
af_speechiness,4082.0,0.000037,9.914065e-01,-5.199338,-0.677141,0.000342,0.675842,5.199338
af_acousticness,4082.0,0.004234,1.022113e+00,-5.199338,-0.673852,-0.002030,0.678468,5.199338
af_instrumentalness,4082.0,-0.273228,1.682350e+00,-5.199338,-0.673508,-0.000762,0.675625,5.199338


In [71]:
from sklearn.preprocessing import QuantileTransformer

df_para_escalar = df_todo.select_dtypes(['number'])

col_af = [col for col in df_para_escalar if col.startswith('af')]
col_aa = [col for col in df_para_escalar if col.startswith('aa')]

scaler = QuantileTransformer(n_quantiles=100, output_distribution = 'uniform', random_state = 0)
scaled_af = pd.DataFrame(scaler.fit_transform(df_para_escalar.loc[:, col_af]),columns=col_af,index=df_para_escalar.index)
scaler = QuantileTransformer(n_quantiles=100, output_distribution = 'uniform', random_state = 0)
scaled_aa = pd.DataFrame(scaler.fit_transform(df_para_escalar.loc[:, col_aa]),columns=col_aa,index=df_para_escalar.index)

scaled_aa.head()

df_escalado = pd.concat([scaled_af, scaled_aa], axis=1)
df_escalado['genre']=df_todo['genre']

df_escalado.to_pickle("../sources/df_todo_1_Uniform.pickle")

df_escalado.describe().T.head(30)


,count,mean,std,min,25%,50%,75%,max
af_duration_ms,4082.0,0.499949,0.288585,0.0,0.248734,0.500315,0.749469,1.0
af_popularity,4082.0,0.413886,0.371783,0.0,0.000000,0.505051,0.747475,1.0
af_danceability,4082.0,0.499944,0.288671,0.0,0.250194,0.499890,0.748990,1.0
af_energy,4082.0,0.500007,0.288791,0.0,0.250000,0.499369,0.750361,1.0
af_key,4082.0,0.495759,0.301745,0.0,0.267677,0.479798,0.737374,1.0
af_loudness,4082.0,0.499943,0.288647,0.0,0.249919,0.500350,0.750611,1.0
af_mode,4082.0,0.644047,0.478860,0.0,0.000000,1.000000,1.000000,1.0
af_speechiness,4082.0,0.500019,0.288623,0.0,0.249158,0.500137,0.750430,1.0
af_acousticness,4082.0,0.500171,0.289000,0.0,0.250203,0.499190,0.751263,1.0
af_instrumentalness,4082.0,0.496636,0.294146,0.0,0.250312,0.499696,0.750361,1.0


In [21]:
df_para_escalar = df_todo.select_dtypes(['number'])

quantil=100

df_todo_quantiles_uniform = pd.DataFrame(
    quantile_transform(
        df_para_escalar,
        n_quantiles = quantil,
        output_distribution = 'uniform', 
        random_state = 0, 
        copy = True
    ), columns=df_para_escalar.columns,index=df_para_escalar.index
)
df_todo_quantiles_uniform['genre']=df_todo['genre']

df_todo_quantiles_uniform.to_pickle("../sources/df_todo_3_UniformScaled.pickle")

df_todo_quantiles_uniform.describe().T.head(30)

,count,mean,std,min,25%,50%,75%,max
duration_ms,4082.0,0.499949,0.288585,0.0,0.248734,0.500315,0.749469,1.0
popularity,4082.0,0.413886,0.371783,0.0,0.000000,0.505051,0.747475,1.0
danceability,4082.0,0.499944,0.288671,0.0,0.250194,0.499890,0.748990,1.0
energy,4082.0,0.500007,0.288791,0.0,0.250000,0.499369,0.750361,1.0
key,4082.0,0.495759,0.301745,0.0,0.267677,0.479798,0.737374,1.0
loudness,4082.0,0.499943,0.288647,0.0,0.249919,0.500350,0.750611,1.0
mode,4082.0,0.644047,0.478860,0.0,0.000000,1.000000,1.000000,1.0
speechiness,4082.0,0.500019,0.288623,0.0,0.249158,0.500137,0.750430,1.0
acousticness,4082.0,0.500171,0.289000,0.0,0.250203,0.499190,0.751263,1.0
instrumentalness,4082.0,0.496636,0.294146,0.0,0.250312,0.499696,0.750361,1.0


In [27]:
df_para_escalar = df_todo.select_dtypes(['number'])

quantil=100

from sklearn.preprocessing import PowerTransformer

transformer = PowerTransformer()

df_todo_quantiles_uniform = pd.DataFrame(
    transformer.fit_transform(
        df_para_escalar
    ), columns=df_para_escalar.columns,index=df_para_escalar.index
)
df_todo_quantiles_uniform['genre']=df_todo['genre']

df_todo_quantiles_uniform.to_pickle("../sources/df_todo_4_RobustScaled.pickle")

df_todo_quantiles_uniform.describe().T.head(30)

C:\Users\Faolin\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


,count,mean,std,min,25%,50%,75%,max
duration_ms,4082.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
popularity,4082.0,-3.329037e-17,1.000123,-1.120897,-1.120897,0.387648,0.912713,1.560214
danceability,4082.0,2.548454e-17,1.000123,-2.419822,-0.781548,0.054466,0.750650,2.666903
energy,4082.0,-3.613528e-16,1.000123,-1.598981,-0.983773,0.064734,1.016628,1.326398
key,4082.0,-4.115060e-17,1.000123,-1.649717,-0.834185,0.067935,0.814126,1.474227
loudness,4082.0,-6.032928e-16,1.000123,-2.571027,-0.777504,0.052242,0.766778,4.327777
mode,4082.0,-1.034123e-15,1.000123,-1.345125,-1.345125,0.743426,0.743426,0.743426
speechiness,4082.0,-4.211341e-16,1.000123,-4.644620,-0.759342,-0.254805,0.683323,2.078511
acousticness,4082.0,-4.966086e-16,1.000123,-1.053993,-1.045835,-0.270137,1.140869,1.263315
instrumentalness,4082.0,-1.195842e-15,1.000123,-1.045863,-1.042498,-0.239238,1.134078,1.353365
